In [183]:
import re

#### 1-Eliminate implication

In [184]:
#done
def E_I(exp):
    pat = r'(\b\w+\b)\s*->\s*(\b\w+\b)'
    return re.sub(pat, r' ~\1 | \2 ', exp)

logical_expression = "(P -> Q)"
eliminated_expression = E_I(logical_expression)
print("Original expression:", logical_expression)
print("Expression with implications eliminated:", eliminated_expression)


Original expression: (P -> Q)
Expression with implications eliminated: ( ~P | Q )


####  3-Remove double-not.

In [230]:
def remove_doube(exp):
  result = []
  i = 0
  while i < len(exp):
    # Check for double negation (~~)
    if exp[i:i+2] == '~~':
      # Skip over the double negation
      i += 2
    else:
      # Append the character to the result
      result.append(exp[i])
      i += 1
  return ''.join(result)

# Example usage
logical_expression = "~~(P & Q) "
expression_without_double_negation = remove_doube(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)


Original expression: ~~(P & Q) 
Expression without double negation: (P & Q) 


#### 2-Move negation inward (Demorgan Law)

In [233]:
def if_quantis(exp):
    final_expression = ""
    counter = 0
    # Loop through each character in the expression
    while counter < len(exp):
        # check if the current position start with negation forall  or negation there exist 
        if exp.startswith("~∀", counter) or exp.startswith("~∃", counter):
            # Extract the quantifier type (opposite of the negated one)
            quanti = '∃' if exp[counter + 1] == '∀' else '∀'
            # Add the extracted quantifier to the final expression
            final_expression += quanti
            # Move the counter past the negation and quantifier symbols
            counter += 2
            # Add the next character (variable name) and a negation symbol
            variable = exp[counter]
            final_expression += variable + "~"
            counter += 1
        else:
            # If not a negated quantifier, simply add the current character
            final_expression += exp[counter]
            counter += 1
    return final_expression

def MNI(expression):
    if expression.find('~') == 2:
        expression = remove_doube(expression)
    # Apply quantifier manipulation
    expression = if_quantis(expression)
    
    # Check if the expression starts with a negation
    if expression.startswith('~'):
        # Remove the negation symbol (~) from the expression
        expression = expression[1:]
        # Negate the entire expression
        expression = "~(" + expression + ")"
    
    # Loop until all negations (~) have been moved inward
    while True:
        # Find the start index of the next negation (~)
        negation_start = expression.find('~(')
        # If no negation is found, exit the loop
        if negation_start == -1:
            break
        
        # Find the corresponding closing parenthesis for the negation
        open_paren_count = 1
        negation_end = negation_start + 2
        for i in range(negation_start + 2, len(expression)):
            if expression[i] == '(':
                open_paren_count += 1
            elif expression[i] == ')':
                open_paren_count -= 1
                if open_paren_count == 0:
                    negation_end = i
                    break
        
        # Extract the inner formula enclosed by the negation
        inner_formula = expression[negation_start + 2:negation_end]
        
        # Split the inner formula by the connective symbols (| or &)
        subExp = inner_formula.split('|') if '|' in inner_formula else inner_formula.split('&')
        
        # Negate each subformula and reconstruct the inner formula
        negated_subformulas = ['~' + subformula if subformula[0] != '~' else subformula[1:] for subformula in subExp]
        # Determine the new logical connective based on the original inner formula
        new_connective = '&' if '|' in inner_formula else '|'
        inner_formula = '(' + new_connective.join(negated_subformulas) + ')'
        
        # Replace the negated expression with the transformed inner formula
        expression = expression[:negation_start] + inner_formula + expression[negation_end + 1:]
    
    # Return the expression with all negations moved inward
    return expression

# Example usage:
# formula = "ّّ~~∀x(~(P(x) & Q(x)) | ~R(x))"
# formula1 = "~∀x(P(x) & Q(x))"
# moved_formula = MNI(formula)
# print("Original formula:", formula)
# print("Formula with negation moved inward:", moved_formula)
# print("=======================================")
# moved_formula1 = MNI(formula1)
# print("Original formula1:", formula1)
# print("Formula with negation moved inward:", moved_formula1)
# print("=======================================")
# formula2 = "~∃x(~P(x) | Q(x))"
# moved_formula2 = MNI(formula2)
# print("Original formula2:", formula2)
# print("Formula with negation moved inward:", moved_formula2)
# print("=======================================")
# formula3 = "~∃x∀y(P(x, y) & Q(x, y))"
# moved_formula3 = MNI(formula3)
# print("Original formula3:", formula3)
# print("Formula with negation moved inward:", moved_formula3)
# print("=======================================")



Original formula: ّّ~~∀x(~(P(x) & Q(x)) | ~R(x))
Formula with negation moved inward: ّّ∀x((~P(x) |~ Q(x)) | ~R(x))
Original formula1: ~∀x(P(x) & Q(x))
Formula with negation moved inward: ∃x(~P(x) |~ Q(x))
Original formula2: ~∃x(~P(x) | Q(x))
Formula with negation moved inward: ∀x(P(x) &~ Q(x))
Original formula3: ~∃x∀y(P(x, y) & Q(x, y))
Formula with negation moved inward: ∀x~∀y(P(x, y) & Q(x, y))


####  4-Standardize variable scope

In [218]:
def SVS(exp):
    l = []
    for expr in exp:
        newExpr = expr
        vars = set()
        rep_map = {}
        def varNew(varOld):
            if varOld in rep_map:
                return rep_map[varOld]
            new = chr(ord('a') + len(vars))
            while new in vars:
                new = chr(ord(new) + 1)
            vars.add(new)
            rep_map[varOld] = new
            return new

        counter = 0
        while counter < len(newExpr):
            if newExpr[counter] in "∀∃":
                var = newExpr[counter + 1]
                if var.islower(): # Confirm variable
                    new_var = varNew(var)
                    newExpr = newExpr[:counter + 1] + newExpr[counter + 1:].replace(var, new_var)
                    counter += len(new_var) - 1
            counter += 1
        l.append(newExpr)

    return l

# Test case
# Test case 1
expressions_1 = ['∃x(P(x) & Q(x)) '  ' ∀x(R(x) | S(x))']
print("Original Expressions 1:", expressions_1)
print("SVS Result 1:", SVS(expressions_1))


# Test case 3
expressions_2 = ['∃x∀y(P(x, y) & Q(x, y))' '∀z∃w(R(z, w) | S(z, w))']
print("\nOriginal Expressions 3:", expressions_2)
print("SVS Result 3:", SVS(expressions_2))


Original Expressions 1: ['∃x(P(x) & Q(x))  ∀x(R(x) | S(x))']
SVS Result 1: ['∃a(P(a) & Q(a))  ∀b(R(b) | S(b))']

Original Expressions 3: ['∃x∀y(P(x, y) & Q(x, y))∀z∃w(R(z, w) | S(z, w))']
SVS Result 3: ['∃a∀b(P(a, b) & Q(a, b))∀c∃d(R(c, d) | S(c, d))']


#### 5- The prenex form (obtained by moving all quantifiers to the left of the
formula.)

In [66]:
#done
def penex_form(exp):
    quanti_part = ""
    parts = ""
    i = 0
    while i < len(exp):
        if exp[i] == '∀' or exp[i] == '∃':
            quanti_part += exp[i] + exp[i+1] + ' '
            i += 1  # Skip the next character which is the variable
        else:
            parts += exp[i]
        i += 1
    return quanti_part.strip() + " " + parts.strip()

# Test case
expression = '∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))'
print("Original Expression:", expression)
print("Penex Form:", penex_form(expression))


Original Expression: ∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))
Penex Form: ∃x ∀y (P(x) ∧ Q(x)) ∧ (R(y) ∨ S(y))


#### 6-Skolemization for existential quantifiers

In [226]:
def skolemization(exp):
    part = exp.split('∃')
    parts_skol = [part[0]]
    for part in part[1:]:
        if part.strip():
            var, i, exp_in = part.partition('(')
            var += '('  # Adding back the opening parenthesis
            
            # Create Skolem function with parentheses
            func_skol = "f" + "("+var.replace('(', '').replace(')', ')')  + ")" # Add closing parenthesis
            
            # Replace variable x in the inner expression with the Skolem function
            skolemized_inner_exp = exp_in.replace('x',  func_skol )
            
            
            # Append skolemized inner expression to the list of skolemized parts
            parts_skol.append(skolemized_inner_exp)
    return ''.join(parts_skol)

# Example usage:
logical_expression = "∃x (P(x) & Q(x))"
skolemized_expression = skolemization(logical_expression)
print("Original expression:", logical_expression)
print("Skolemized expression:", skolemized_expression)


Original expression: ∃x (P(x) & Q(x))
Skolemized expression: P(f(x )) & Q(f(x )))


#### 7-Eliminate universal quantifiers.

In [227]:
# done
def E_U(exp):
    # Define a pattern to match universal quantifiers and their corresponding variables and inner expressions
    patrn = r'∀(\w+)\s*\((.*?)\)'

    # Use regex to remove all occurrences of universal quantifiers
    withoutForAll = re.sub(patrn, r'(\2)', exp)

    return withoutForAll

# Example usage:
logical_expression = "∀x (P(x) & Q(x))"
expression_without_universal_quantifiers = E_U(logical_expression)
print("Original expression:", logical_expression)
print("Expression without universal quantifiers:", expression_without_universal_quantifiers)


Original expression: ∀x (P(x) & Q(x))
Expression without universal quantifiers: (P(x) & Q(x))


#### 8-Convert to conjunctive normal form

In [179]:
""" BIDIRECTIONAL ELIMINATION """
def bidirectional_elimination(components):
    # Find all occurrences of "iff" in the components list
    if "<->" in components:
        index = components.index("<->")  # Find the index of "iff"
        literal_1, literal_2 = components[index + 1], components[index + 2]
        # Replace "iff" and its operands with equivalent implications
        components[index:index + 3] = ["&", ["->", literal_1, literal_2], ["->", literal_2, literal_1]]

    # Recursively apply bidirectional elimination to sub-components if present
    for index, literal in enumerate(components):
        if isinstance(literal, list):  # Check if the literal is a compound statement
            bidirectional_elimination(literal)


['and', ['or', 'A', ['iff', 'B', 'C']], ['iff', 'D', ['or', 'E', 'F']]]


In [229]:
def apply_distributive_law(expression):
    if expression[0] == "|":
        if len(expression[2]) > 1 and expression[2][0] == "&":
            sub_expression_1 = expression[1]
            sub_expression_2 = expression[2]
            del expression[:]
            expression.append("&")
            for sub_exp in sub_expression_2[1:]:
                if sub_exp == "&":
                    continue
                expression.append(["|", sub_expression_1, sub_exp])
        elif len(expression[1]) > 1 and expression[1][0] == "&":
            sub_expression_1 = expression[1]
            sub_expression_2 = expression[2]
            del expression[:]
            expression.append("&")
            for sub_exp in sub_expression_1[1:]:
                if sub_exp == "&":
                    continue
                expression.append(["|", sub_exp, sub_expression_2])

    for sub_exp in expression:
        if len(sub_exp) > 1:
            apply_distributive_law(sub_exp)



Test Case 1: ['or', ['and', 'A', 'B'], 'C']
Test Case 2: ['or', 'A', ['and', 'B', 'C']]


In [ ]:
def convert_to_CNF(expression):
    # step 1: Eliminate BIDIRECTIONAL
    expression = bidirectional_elimination(expression)
    # Step 2: Eliminate Implications
    expression = E_I(expression)
    
    # Step 3: Move Negation Inward
    expression = MNI(expression)
    
    # Step 4: Remove Double Negations
    expression = remove_doube(expression)
    
    # Step 5: Standardize Variable Scope
    expression = SVS(expression)
    
    # Step 6: Move Universal Quantifiers Outward
    expression = E_U(expression)
    
    # Step 7: Skolemization
    expression = skolemization(expression)
    
    # Step 8: Convert to CNF
    expression = convert_CNF(expression)
    
    return expression

# Example usage:
logical_expression = "∀x (P(x) & Q(x))"
cnf_expression = convert_to_CNF(logical_expression)
print("Original expression:", logical_expression)
print("Expression in CNF:", cnf_expression)


In [174]:
import sys
import copy
import os.path

""" CONVERT TO CNF """
def convert_CNF(sentence):
    components=eval(sentence)    #convert string to list
    bidirectional_elimination(components)
    implication_elimination(components)
    demorgans(components)
    distributivity(components)  
    flag=1
    while(components[0]=="or" and flag==1):
        if (len(components[2])>1  and components[2][0]=="and"):
            distributivity_recursion(components)
            flag=1
        elif (len(components[1])>1  and components[1][0]=="and"):
            distributivity_recursion(components)
            flag=1
        else:
            flag=0

    associvity(components)
    remove_parens(components)
    remove_duplicates(components)
    remove_single_literals(components)
    quality_check(components)
    remove_parens(components)

    check=1
    if(components[0]=="and"):
       temp_literal=components[1]
       for x in range(1,len(components)):
           if temp_literal != components[x]:
               check=0

    if check==1:
        remove_duplicates(components)
        remove_single_literals(components)
        quality_check(components)
        remove_parens(components)
    return components

""" BIDIRECTIONAL ELIMINATION"""
def bidirectional_elimination(components): 
    if(components[0]=="iff"):
        literal_1=components[1]
        literal_2=components[2]
        components[0]="and"
        components[1]=["implies",literal_1,literal_2]
        components[2]=["implies",literal_2,literal_1]

    for literal in components:
        if(len(literal)>1):
            bidirectional_elimination(literal)

""" IMPLICATION ELIMINATION"""
def implication_elimination(components):  
    if(components[0]=="implies"):
        literal_1=components[1]
        components[0]="or"
        components[1]=["not",literal_1]

    for literal in components:
        if(len(literal)>1):
            implication_elimination(literal)
 
""" DE-MORGAN'S LAW """
def demorgans(components): 
    if(components[0]=="not"):
        literal=components[1]

        if(literal[0]=="not"):
            del components[:]
            if(len(literal[1])==1):
                components.append(literal[1])
            else:
                for literal_1 in literal[1]:
                    components.append(literal_1)

                if(len(components)>1):
                    demorgans(components)

        elif(literal[0]=="or"):
            del components[:]
            components.append("and")
            for literal_1 in literal:
                if literal_1== "or":
                    continue
                else:
                    components.append(["not",literal_1])

        elif(literal[0]=="and"):
            del components[:]
            components.append("or")
            for literal_1 in literal:
                if literal_1== "and":
                    continue
                else:
                    components.append(["not",literal_1])
        
       

    for literal in components:
        if(len(literal)>1 ):
            demorgans(literal)

""" DISTRIBUTIVE LAW """
def distributivity(components):
    if(components[0]=="or"):
        if(len(components[2])>1  and components[2][0]=="and"):
            literal_1=components[1]
            literal_2=components[2]
            del components[:]
            components.append("and")
            for literal_x in literal_2:
                if literal_x=="and":
                    continue
                components.append(["or",literal_1,literal_x])
        elif(len(components[1])>1  and components[1][0]=="and"):
            literal_1=components[1]
            literal_2=components[2]
            del components[:]
            components.append("and")
            for literal_x in literal_1:
                if literal_x=="and":
                    continue
                components.append(["or",literal_x,literal_2])

    for literal in components:
        if(len(literal)>1 ):
            distributivity(literal)

""" OUTER RECURSIVE DISTRIBUTIVE LAW"""
def distributivity_recursion(components):
    if(components[0]=="or"):
        if(len(components[2])>1  and components[2][0]=="and"):
            literal_1=components[1]
            literal_2=components[2]
            del components[:]
            components.append("and")
            for literal_x in literal_2:
                if literal_x=="and":
                    continue
                components.append(["or",literal_1,literal_x])
        elif(len(components[1])>1  and components[1][0]=="and"):
            literal_1=components[1]
            literal_2=components[2]
            del components[:]
            components.append("and")
            for literal_x in literal_1:
                if literal_x=="and":
                    continue
                components.append(["or",literal_x,literal_2])

""" ASSOCIATIVE LAW """
def associvity(components):
    if(components[0]=="and"):
        temp_components=copy.deepcopy(components)
        del components[:]
        for literal in temp_components:
            if literal=="and":
                components.append("and")
            else:
                if literal[0]!="and":
                    components.append(literal)
                else:
                    for literal_x in literal:
                        if literal_x=="and":
                            continue
                        else:
                            components.append(literal_x)

    if(components[0]=="or"):
        temp_components=copy.deepcopy(components)
        del components[:]
        for literal in temp_components:
            if literal=="or":
                components.append("or")
            else:
                if literal[0]!="or":
                    components.append(literal)
                else:
                    for literal_x in literal:
                        if literal_x=="or":
                            continue
                        else:
                            components.append(literal_x)

    for literal in components:
        if(len(literal)>1):
            associvity(literal) 
                   
""" REMOVE PARENS """   
def remove_parens(components):
     i=0
     for literal in components:
        if(len(literal)==1):
            temp=str(literal)  
            if(len(temp)>1):
              components[i]=temp[2]
        elif(len(literal)>1):
            remove_parens(literal)
        i=i+1

""" REMOVE DUPLICATES """
def remove_duplicates(components):
    delete_literals=[]
    if(components[0]=="and" or components[0]=="or"):
        for x in range(0,len(components)):
            for y in range(x+1,len(components)):
                literal_1=sorted(components[x])
                literal_2=sorted(components[y])
                if literal_1==literal_2:
                   delete_literals.append(components[y])
 
    if(len(delete_literals)>=1):
        temp_components=[]
        for literal in delete_literals:
            flag=1
            for literal_x in components:
                if (literal==literal_x and flag==1):
                    flag=0
                    continue
                else:
                    temp_components.append(literal_x)
        
        del components[:]
        for literal in temp_components:
            components.append(literal)
        

    for literal in components:
        if(len(literal)>1):
            remove_duplicates(literal) 

""" REMOVE SINGLE LITERALS WITH AND-OR """
def remove_single_literals(components):
    if(len(components)==2 and components[0]!="not"):
        del components[0]

    for literal in components:
        if(len(literal)>1 and isinstance(literal, list)):
            remove_single_literals(literal)

""" QUALITY CHECK """
def quality_check(components):
    if(len(components)==1 and len(components[0])>1):
        literal=components[0]
        del components[:]
        for literal_x in literal:
            components.append(literal_x)

    for literal in components:
        if(len(literal)>1):
            quality_check(literal)

   
""" WRITE TO OUTPUT FILE """
def writeOutput(final_list):
    filename='sentences_CNF.txt'
    outputFile = open(filename,'a')

    for sentence in final_list:
        sentence=str(sentence)
        sentence=sentence.replace("\'","\"")
        outputFile.write(sentence)
        outputFile.write("\n")
    
    outputFile.close()

""" CLEAR THE OUTPUT FILE """
def clear():
    filename='sentences_CNF.txt'
    outputFile = open(filename,'w')
    outputFile.truncate()
    outputFile.close()

""" READ THE INPUT FILE """
clear()
inputFile = open(sys.argv[2])
conf=0
line_number=0
final_list=[]
for line in inputFile:
    if conf==0:
        spl = line.strip().split(' ')
        line_number=int(spl[0])
        conf=1
    else:
        sentence=convert_CNF(line)
        if(len(sentence)==1):
            temp="\""+sentence[0]+"\""
            final_list.append(temp)
        else:
            final_list.append(sentence)
                    
inputFile.close()
writeOutput(final_list)

IndexError: list index out of range